<a href="https://colab.research.google.com/github/SonOf1998/ProblemSet4/blob/main/ps4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First install the package that makes it easy to get the required images.
Unfortunately it implicitly installs tensorflow-cpu and it makes the google colab doing its computations on CPU even if I set the runtime type! Uninstalling and reinstalling tensorflow in an additional cell solves the problem.



In [ ]:
!pip install openimages

In [ ]:
!pip uninstall --yes tensorflow-cpu
!pip uninstall --yes tensorflow
!pip install tensorflow

In [3]:
from openimages.download import download_images
import os
import shutil

In [4]:
# removes every directory from the directory given in the parameter
def clear_workdir(workdir):
  for filename in os.listdir(workdir):
    filepath = os.path.join(workdir, filename)
    if os.path.isdir(filepath):
      shutil.rmtree(filepath)

# creates empty directories for training, validation and testing data
def make_set_directory(set_name, classes):
  os.mkdir(set_name)
  for cls in classes:
    os.mkdir(os.path.join(set_name, cls))

The download_images() function download 600 pictures of each category to the /[category]/images folder. We'd like to move these pictures into folders reserved for training/validation/testing for each category.

To make the file structure transparent (easy to see through) I also remove the unneeded folders after moving all the pictures to the proper location. 

In [5]:
workdir = os.getcwd()
clear_workdir(workdir)

# These are the classes I selected for the exercise..
# For whatever reason download_images() fails if I don't
# use upper case for the initial letter of the class' strings
classes = ["Car", "Bus", "Train"]
download_images(workdir, classes, exclusions_path=None, limit=600)

# Converts class strings to lowercase letters
# as download_images() make dirs with only lowercase names
for i in range(len(classes)):
  classes[i] = classes[i].lower()

set_dirs = ["training", "validation", "testing"]
for set_dir in set_dirs:
  make_set_directory(set_dir, classes)

nb_training = 400
nb_validation = 100
nb_testing = 100

for cls in classes:
  path_to_class = os.path.join(cls, "images")
  for i, filename in enumerate(os.listdir(path_to_class)):
    full_path_to_pic = os.path.join(path_to_class, filename)
    if i < nb_training:
      shutil.move(full_path_to_pic, os.path.join(workdir, set_dirs[0], cls, filename))
    elif i < nb_training + nb_validation:
      shutil.move(full_path_to_pic, os.path.join(workdir, set_dirs[1], cls, filename))
    else:
      shutil.move(full_path_to_pic, os.path.join(workdir, set_dirs[2], cls, filename))
  
  # we moved every picture to our train/valid/test set
  # so we can delete the empty directory
  shutil.rmtree(os.path.join(workdir, cls))



2020-11-08  18:58:25 INFO NumExpr defaulting to 2 threads.
2020-11-08  18:58:27 INFO Downloading 600 train images for class 'car'
100%|██████████| 600/600 [00:13<00:00, 43.90it/s]
2020-11-08  18:58:41 INFO Downloading 600 train images for class 'bus'
100%|██████████| 600/600 [00:13<00:00, 44.14it/s]
2020-11-08  18:58:55 INFO Downloading 600 train images for class 'train'
100%|██████████| 600/600 [00:13<00:00, 44.62it/s]


In [6]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ModelCheckpoint, History, EarlyStopping

As loading every picture would probably cause memory problems in colab I used generators for training and testing. As a **preprocessing function** I used the one I found in the inception_v3 module, hopefully correctly.  

At the end of this cell as an output we can see that our training set, validation set and training set contains 3x400, 3x100, 3x100 images respectively.

In [7]:
img_height = 256     # Input image height
img_width  = 256     # Input image width
batch_size = 32
class_mode = 'categorical'
color_mode = 'rgb' 

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_iter = train_datagen.flow_from_directory(
    os.path.join(workdir, set_dirs[0]),
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = class_mode,
    color_mode = color_mode
)

validation_iter = train_datagen.flow_from_directory(
    os.path.join(workdir, set_dirs[1]),
    target_size=(img_height, img_width),
    batch_size = batch_size,
    class_mode = class_mode,
    color_mode = color_mode
)

test_iter = test_datagen.flow_from_directory(
    os.path.join(workdir, set_dirs[2]),
    target_size=(img_height, img_width),
    batch_size = 1,
    class_mode = class_mode,
    color_mode = color_mode
)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


Creating the base model from the InceptionV3 template, with weights initialized according to imagenet.

I added a custom fully connected layer to the end.

Additionally, I defined two callbacks: one for model saving and one for early stopping for the case when the training fails to improve validation accuracy.

In [8]:
base_model = InceptionV3(input_shape=(img_height, img_width, 3),
                    weights="imagenet",
                    include_top=False,
                    classes=3)

inputs = Input(shape=(img_height, img_width, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(32, activation="relu")(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(32, activation="relu")(x)
outputs = Dense(3, activation="softmax")(x)

model = Model(inputs, outputs)

estopping = EarlyStopping(monitor="val_acc", patience=7, verbose=1)
checkpoint = ModelCheckpoint("chk.chk", save_weights_only=True, save_best_only=True, monitor="val_acc", verbose=1)

87916544/87910968 [==============================] - 0s 0us/step


Train the model's fully connected part by disabling the **base_model**.

In [9]:
for layer in base_model.layers:
  layer.trainable = False

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc"])

model.fit(
    train_iter,
    epochs=30,
    validation_data=validation_iter,
    callbacks=[checkpoint, estopping]) 

Epoch 1/30
38/38 [==============================] - ETA: 0s - loss: 0.8868 - acc: 0.6150
Epoch 00001: val_acc improved from -inf to 0.91000, saving model to chk.chk
38/38 [==============================] - 40s 1s/step - loss: 0.8868 - acc: 0.6150 - val_loss: 0.5184 - val_acc: 0.9100
Epoch 2/30
38/38 [==============================] - ETA: 0s - loss: 0.5004 - acc: 0.8467
Epoch 00002: val_acc improved from 0.91000 to 0.92333, saving model to chk.chk
38/38 [==============================] - 38s 1s/step - loss: 0.5004 - acc: 0.8467 - val_loss: 0.3077 - val_acc: 0.9233
Epoch 3/30
38/38 [==============================] - ETA: 0s - loss: 0.3861 - acc: 0.8800
Epoch 00003: val_acc improved from 0.92333 to 0.92667, saving model to chk.chk
38/38 [==============================] - 38s 999ms/step - loss: 0.3861 - acc: 0.8800 - val_loss: 0.2559 - val_acc: 0.9267
Epoch 4/30
38/38 [==============================] - ETA: 0s - loss: 0.3415 - acc: 0.8858
Epoch 00004: val_acc did not improve from 0.92667


Now, as we barely see any improvement and has relatively high validation accuracy we can enable training on the InceptionV3 segment (at least on a fraction of it as the exercised asked) without worrying about totally messing up the imagenet's precalculated weights.

I make only the upper half of the inception base model trainable.

Eventually we load back the best saved model

In [10]:
inception_layer_cnt = len(base_model.layers)
for i in range(inception_layer_cnt // 2, inception_layer_cnt):
  base_model.layers[i].trainable = True

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc"])

model.fit(
    train_iter,
    epochs=30,
    validation_data=validation_iter,
    callbacks=[checkpoint, estopping]) 

model.load_weights("chk.chk")

Epoch 1/30
38/38 [==============================] - ETA: 0s - loss: 0.4843 - acc: 0.8358
Epoch 00001: val_acc did not improve from 0.95000
38/38 [==============================] - 41s 1s/step - loss: 0.4843 - acc: 0.8358 - val_loss: 0.2618 - val_acc: 0.9100
Epoch 2/30
38/38 [==============================] - ETA: 0s - loss: 0.2617 - acc: 0.9108
Epoch 00002: val_acc did not improve from 0.95000
38/38 [==============================] - 38s 1s/step - loss: 0.2617 - acc: 0.9108 - val_loss: 0.2192 - val_acc: 0.9233
Epoch 3/30
38/38 [==============================] - ETA: 0s - loss: 0.2121 - acc: 0.9258
Epoch 00003: val_acc did not improve from 0.95000
38/38 [==============================] - 38s 1s/step - loss: 0.2121 - acc: 0.9258 - val_loss: 0.3417 - val_acc: 0.9100
Epoch 4/30
38/38 [==============================] - ETA: 0s - loss: 0.2119 - acc: 0.9283
Epoch 00004: val_acc did not improve from 0.95000
38/38 [==============================] - 38s 1s/step - loss: 0.2119 - acc: 0.9283 - val

Let's see how our model performs now on the test set.  
We interested in the returned accuracy value.

In [11]:
loss, acc = model.evaluate(test_iter)
print(acc)

300/300 [==============================] - 6s 19ms/step - loss: 0.2110 - acc: 0.9500
0.949999988079071
